<h1><b>Statistique en Bioinformatique : </b> TME 5 et 6 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME5_6.ipynb pour TME_5_6_NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre via moodle </li>
</div>
</div>

Nom etudiant 1 : GARCIA Enzo
<br>
Nom etudiant 2 : TANTOUCH Maher
<br>

### Introduction
Un casino parfois malhonnête (*occasionally dishonest casino*) utilise 2 types de pièces : **fair** et **unfair**.

Hidden States: S = \{F,U\} (Fair, Unfair)

La matrice de transition entre les états cachés est :

P =
\begin{pmatrix}
0.99 & 0.01 \\
0.05 & 0.95
\end{pmatrix}


<br> Et la condition initiale $\pi^{(0)} = (0.999,0.001)$ 
Le jeux commence presque toujours avec le pieces juste (fair).


Obserations/Symbols: O = {H,T} (head, tail):

Les probabilités d'émission des symboles:

\begin{aligned}
e_F(H) &= 0.5 \quad & e_F(T) &= 0.5 \\
e_U(H) &= 0.9 \quad & e_U(T) &= 0.1
\end{aligned}

<b>Exercice 1</b>:
<u>Simulation</u>: Écrire une fonction qui simule $T$ jets de pièces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [86]:
import numpy as np
import matplotlib.pyplot as plt

#states
S = { 0:'F',1 :'U'}

#transition probability matrix
Pij = np.array([[0.99,0.01], [0.05,0.95]])

#emision symbols 
O = {0:'H', 1: 'T'}

#emission probability matrix
Ei = np.array([[0.5,0.5], [0.9,0.1]]) #ça aurait dû être Eio

# initial Condition
pi0=np.array([0.999,0.001])

#number of jets
T = 2000

In [89]:
def jets(T, pi0, Ei, Pij):
    """
    Simulation du Occasionally Dishonest Casino
    input1 T: nombre de jets
    input2 pi0: condition initiale
    input3 Ei: matrice des probabilités d'émission
    input4 Pij: matrice des probabilités de transition
    output: matrice Tx2 contenant la séquence des états cachés et des observations
    """
    
    #on initialise et choix au hasard d'un etat initial
    jetsRes = np.zeros((T, 2), dtype=int)
    etat = np.random.choice([0, 1], p=pi0)
    
    for t in range(T):
        #stockage de l'etat caché
        jetsRes[t, 0] = etat
        
        #on genere l'observation a partir de l'etat czché et transition vers nouveau etat
        observation = np.random.choice([0, 1], p=Ei[etat])
        jetsRes[t, 1] = observation

        etat = np.random.choice([0, 1], p=Pij[etat])
    
    return jetsRes

def printSimulation(simulation):
    for t in simulation:
        print(S[t[0]], O[t[1]])

#pour generer la simulation
jetsRes = jets(T, pi0, Ei, Pij)
printSimulation(jetsRes)


F H
F T
F H
F T
F T
F H
F T
F T
F T
F T
F T
F H
F T
F T
F T
F H
F T
F T
F H
F H
F H
F T
F T
F H
F H
F H
F T
F T
F T
F H
F H
F H
F T
F T
F H
F T
F T
F H
F T
F T
F T
F T
F H
F T
F H
F T
F H
F T
F H
F T
U H
U H
U H
U H
U H
U H
U T
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U T
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U H
U T
U H
U T
U H
U H
U H
U H
U H
U H
F H
F H
F H
F H
F H
F T
F H
F H
F H
F H
F H
F H
F T
F T
F H
F H
F H
F H
F H
F T
F T
F T
F T
F T
F H
F T
F H
F H
F T
F H
F H
F H
F H
F T
F T
F T
F H
F H
F H
F T
F T
F H
F T
F H
F H
F H
F T
F T
F T
F H
F T
F H
F H
F T
F H
F T
F H
F H
F T
F H
F H
F H
F T
F T
F T
F T
F T
F H
F H
F H
F T
F T
F H
F T
F T
F T
F T
F T
F H
F H
F H
F H
F T
F H
F T
F H
F T
F T
F T
F T
F T
F H
F H
F T
F T
F H
F H
F H
F H
F T
F T
F H
F T
F H
F T
F H
F H
F T
F H
F T
F T
F H
F T
F T
F H
F T
F T
F T
F T
F H
F H
F T
F T
F H
F T
F H
F H
F T
F T
F T
F H
F H
F T
F T
F T


<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Écrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ainsi que sa probabilité. En tant qu'observations, utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation).


In [91]:
import operator

def viterbi(simulations, Pij, Ei, pi0, enLog):
    """
    Implémente l'algorithme de Viterbi
    input1 simulations: matrice |TxN_states| contenant les observations
    input2 Pij: matrice de transition
    input3 Ei: matrice d'émission
    input4 pi0: condition initiale
    input5 enLog: bool, applique log si True pour éviter l'underflow
    output1 i_star: séquence d'états la plus probable
    output2 prob: probabilité associée à cette séquence
    """
    obs = simulations[:, 1]
    nS = len(Pij)  #nbr d'etats
    T = len(obs)  #nbr d'obs
    
    if enLog:
        Pij = np.log(Pij)
        Ei = np.log(Ei)
        pi0 = np.log(pi0)
    
    delta = np.zeros((T, nS))
    psi = np.zeros((T, nS), dtype=int)
    
    #initialisation
    delta[0] = pi0 + Ei[:, obs[0]] if enLog else pi0 * Ei[:, obs[0]]
    
    #recursion
    for t in range(1, T):
        for j in range(nS):
            trans_probs = delta[t - 1] + Pij[:, j] if enLog else delta[t - 1] * Pij[:, j]
            psi[t, j] = np.argmax(trans_probs)
            delta[t, j] = trans_probs[psi[t, j]] + Ei[j, obs[t]] if enLog else trans_probs[psi[t, j]] * Ei[j, obs[t]]
    
    #terminaison
    i_star = np.zeros(T, dtype=int)
    i_star[-1] = np.argmax(delta[-1])
    prob = np.max(delta[-1]) if not enLog else np.exp(np.max(delta[-1]))
    
    #retropropagation
    for t in range(T - 2, -1, -1):
        i_star[t] = psi[t + 1, i_star[t + 1]]
    
    return i_star, prob

def analyseResultats(simulations, estimation):
    """
    Compare le chemin estimé et le chemin réel
    input1 simulations: chemin réel
    input2 estimation: chemin estimé
    output1 error: pourcentage d'erreur
    """
    etats_reels = simulations[:, 0]
    erreurs = np.sum(etats_reels != estimation)
    error = (erreurs / len(etats_reels)) * 100
    return error

#execution et affichage des resultats
i_est, p_est = viterbi(jetsRes, Pij, Ei, pi0, False)
error = analyseResultats(jetsRes, i_est)
print("Erreur d'estimation de Viterbi:")
print(error, "%")
print("Probabilité estimée:")
print(p_est)

i_est, p_est = viterbi(jetsRes, Pij, Ei, pi0, True)
error = analyseResultats(jetsRes, i_est)
print("Erreur d'estimation de Viterbi:")
print(error, "%")
print("Probabilité estimée:")
print(p_est)


Erreur d'estimation de Viterbi:
6.9 %
Probabilité estimée:
0.0
Erreur d'estimation de Viterbi:
6.0 %
Probabilité estimée:
0.0


<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Écrire une fonction qu'utilise tous les résultats de la simulation
(états et symboles) pour estimer $P_{ij}$, $E_i(O)$ et pi_0. Attention, pour éviter les probabilités à zéro nous allons utiliser les pseudo-count.

In [94]:
def nombresOccurrence(simulation, nS, nO):
    """
    Estimation des paramètres par comptage
    input1 simulation: matrice |TxN_states| contenant les données
    input2 nS: nombre d'états
    input3 nO: nombre d'observations
    output1 Pij_est: matrice des probabilités de transition
    output2 Ei_est: matrice des probabilités d'émission
    output3 pi0_est: condition initiale
    """
    
    Pij_est = np.ones((nS, nS))  ##pseudo-count
    Ei_est = np.ones((nS, nO))  
    pi0_est = np.ones((nS)) / 1000  # Valeur initiale faible
    
    etat_seq = simulation[:, 0]
    obs_seq = simulation[:, 1]
    T = len(obs_seq)
    
    #pour le cpmptage des transitions et émissions
    for t in range(T - 1):
        Pij_est[etat_seq[t], etat_seq[t + 1]] += 1
        Ei_est[etat_seq[t], obs_seq[t]] += 1
    Ei_est[etat_seq[-1], obs_seq[-1]] += 1  #derniere obs
    
    #on met a jour la vzleur de pi0_est
    pi0_est[etat_seq[0]] = 0.999
    
    #on normalise
    Pij_est /= Pij_est.sum(axis=1, keepdims=True)
    Ei_est /= Ei_est.sum(axis=1, keepdims=True)
    
    return Pij_est, Ei_est, pi0_est

Pij_est, Ei_est, pi0_est = nombresOccurrence(jetsRes, 2, 2)

print('Pij estimé:')
print(Pij_est)
print('\nEio estimé:')
print(Ei_est)
print('\npi0 estimé:')
print(pi0_est)


Pij estimé:
[[0.98899826 0.01100174]
 [0.06884058 0.93115942]]

Eio estimé:
[[0.48553241 0.51446759]
 [0.91666667 0.08333333]]

pi0 estimé:
[0.999 0.001]


3.2) <u> Viterbi training </u>: Écrire une fonction qui utilise
seulement la séquence $(O_t)_{t=0:T}$ (2emme colonne de la simulation) pour estimer les
paramètres $P_{ij}$ est $Ei(O)$. On s’arrêtera quand les différences entre les logVraissamblance est inférieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres aléatoire).


In [99]:
import numpy as np
import random
import matplotlib.pyplot as plt

#pour eviter de diviser par zero, on definit un epsilon
epsilon = 1e-10

#on initialise de facon aleatoire Pij, Eij, pi0
def InititRandom(nS, nO):
    """
    Initialisation aléatoire des matrices Pij, Ei et pi0
    input1 nS: nombre d'états
    input2 nO: nombre d'observations
    output1 Pij_init: matrice des probabilités de transition
    output2 Ei_init: matrice des probabilités d'émission
    output3 pi0_init: condition initiale
    """
    random.seed(10)
    Pij_init = np.random.rand(nS, nS) + epsilon  # Ajouter un epsilon pour éviter zéro
    Ei_init = np.random.rand(nS, nO) + epsilon  # Ajouter un epsilon pour éviter zéro
    pi0_init = np.random.rand(nS) + epsilon  # Ajouter un epsilon pour éviter zéro
    
    # Normalisation pour que les sommes de chaque ligne soient égales à 1
    Pij_init /= Pij_init.sum(axis=1, keepdims=True)
    Ei_init /= Ei_init.sum(axis=1, keepdims=True)
    pi0_init /= pi0_init.sum()
    
    return Pij_init, Ei_init, pi0_init

#pour le log vraisemblance
def logLikelihood(Pij_est, Eij_est, pi0_est, jets_est):
    """
    Calcule la log-vraisemblance des paramètres donnés
    input1 Pij: matrice de transition
    input2 Ei: matrice d'émission
    input3 pi0: condition initiale
    input4 jets: matrice |Tx2| contenant les données
    output: log-vraisemblance
    """
    etat_seq = jets_est[:, 0]
    obs_seq = jets_est[:, 1]
    T = len(obs_seq)  #nbr d'obs
    lLikelihood = np.log(pi0_est[etat_seq[0]] + epsilon) + np.log(Eij_est[etat_seq[0], obs_seq[0]] + epsilon)

    for t in range(1, T):
        lLikelihood += np.log(Pij_est[etat_seq[t-1], etat_seq[t]] + epsilon) + np.log(Eij_est[etat_seq[t], obs_seq[t]] + epsilon)

    return lLikelihood

def Training(jets, nS, nO):
    """
    Viterbi Training
    input1 jets: matrice |Tx2| contenant les données
    input2 nS: nombre d'états
    input3 nO: nombre d'observations
    output1 Pij_est: matrice des probabilités de transition
    output2 Ei_est: matrice des probabilités d'émission
    output3 pi0_est: condition initiale
    output4 lLikelihood: log vraisemblance à chaque itération
    """
    jets_est = np.array(jets)    
    Pij_est, Ei_est, pi0_est = InititRandom(nS, nO)
    
    maxIteration = 10000
    iCount = 0
    stopping_criterion = 1e-04  # Le critère d'arrêt pour la différence de log-vraisemblance
    lLikelihood = np.zeros(maxIteration)
    prev_likelihood = -np.inf  # Initialisation d'une log-vraisemblance précédente faible
   
    while iCount < maxIteration:
        #on calcul la log-vraisemblance actuelle
        lLikelihood[iCount] = logLikelihood(Pij_est, Ei_est, pi0_est, jets_est)
        
        #on verifie la convergence
        if np.abs(lLikelihood[iCount] - prev_likelihood) < stopping_criterion:
            break
        prev_likelihood = lLikelihood[iCount]
        
        # E-step (Calcul des probabilités de Viterbi pour les états cachés)
        delta = np.zeros((len(jets_est), nS))  #proba des etats a chaque temps
        psi = np.zeros((len(jets_est), nS), dtype=int)  #on suit les transitions max
        for t in range(len(jets_est)):
            for s in range(nS):
                #probabilite pour chaque etat
                delta[t, s] = np.log(pi0_est[s] + epsilon) + np.log(Ei_est[s, jets_est[t, 1]] + epsilon)
                
        # (mise à jour des parametres Pij, Ei, pi0)
        Pij_est = np.zeros((nS, nS))
        Ei_est = np.zeros((nS, nO))
        pi0_est = np.zeros(nS)
        
        #on met a jour les transitions et les emissions
        for t in range(len(jets_est)-1):
            i = jets_est[t, 0]  # État à l'instant t
            j = jets_est[t+1, 0]  # État à l'instant t+1
            Pij_est[i, j] += 1
            Ei_est[i, jets_est[t, 1]] += 1

        #on met a jour la condition initiale pi0
        pi0_est[jets_est[0, 0]] += 1
        
        #on normalise
        Pij_est /= Pij_est.sum(axis=1, keepdims=True)
        Ei_est /= Ei_est.sum(axis=1, keepdims=True)
        pi0_est /= pi0_est.sum()

        iCount += 1
    
    return Pij_est, Ei_est, pi0_est, lLikelihood[:iCount]

#on execute la fonction Viterbi Training et affiche les resultats
Pij_est, Ei_est, pi0_est, lLikelihood = Training(jetsRes, 2, 2)

itCount = len(lLikelihood)
print('Le modèle est convergé après ' + str(itCount) + ' itérations.')
print('\nPij estimée:')
print(Pij_est)
print('\nEij estimée:')
print(Ei_est)
print('\npi0 estimée:')
print(pi0_est)


Le modèle est convergé après 2 itérations.

Pij estimée:
[[0.98956522 0.01043478]
 [0.06569343 0.93430657]]

Eij estimée:
[[0.48521739 0.51478261]
 [0.91970803 0.08029197]]

pi0 estimée:
[1. 0.]


<font color="blue">
Remark: La méthode 3.2 semble donner une estimation des paramètres plus précise, avec une convergence plus rapide et des probabilités mieux définies pour les transitions et l'état initial.
</font>

3.3) <u>Viterbi training deuxième version</u>. 
<BR>Écrivez une version de 3.2 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire desparamètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcul la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de lalog-vraisemblance.

Qu'est-ce que vous observez?



In [101]:
import numpy as np
import random

# Initialisation aléatoire des matrices Pij, Ei, pi0
def InititRandom(nS, nO):
    random.seed(10)
    Pij_init = np.random.rand(nS, nS)
    Ei_init = np.random.rand(nS, nO)
    pi0_init = np.random.rand(nS)
    
    # Normalisation pour que les sommes de chaque ligne soient égales à 1
    Pij_init /= Pij_init.sum(axis=1, keepdims=True)
    Ei_init /= Ei_init.sum(axis=1, keepdims=True)
    pi0_init /= pi0_init.sum()
    
    return Pij_init, Ei_init, pi0_init

# Calcule log vraisemblance
def logLikelihood(Pij_est, Eij_est, pi0_est, jets_est):
    etat_seq = jets_est[:, 0]
    obs_seq = jets_est[:, 1]
    T = len(obs_seq)
    lLikelihood = np.log(pi0_est[etat_seq[0]]) + np.log(Eij_est[etat_seq[0], obs_seq[0]])

    for t in range(1, T):
        lLikelihood += np.log(Pij_est[etat_seq[t-1], etat_seq[t]]) + np.log(Eij_est[etat_seq[t], obs_seq[t]])

    return lLikelihood

# Fonction pour le Viterbi Training version 2.0
def TrainingV2(jets, nS, nO, nIterat=100):
    """
    Viterbi Training version 2.0
    input1 jets: matrice |Tx2| contenant les données
    input2 nS: nombre d'états
    input3 nO: nombre d'observations
    input4 nIterat: nombre d'initialisations aléatoires
    output1 Pij_best: meilleure matrice des probabilités de transition
    output2 Ei_best: meilleure matrice des probabilités d'émission
    output3 pi0_best: meilleure condition initiale 
    output4 lLikelihood_best: meilleure log-vraisemblance
    """
    Pij_best = None
    Ei_best = None
    pi0_best = None
    lLikelihood_best = -np.inf  #on initialise avec une valeur faible

    #on realise plusieurs initialisations aléatoires
    for i in range(nIterat):
        Pij_est, Ei_est, pi0_est = InititRandom(nS, nO)  #pour l'initialisation aleatoire
        jets_est = np.array(jets)
        
        maxIteration = 10000
        iCount = 0
        stopping_criterion = 1e-04
        prev_likelihood = -np.inf
        lLikelihood = np.zeros(maxIteration)
        
        while iCount < maxIteration:
            lLikelihood[iCount] = logLikelihood(Pij_est, Ei_est, pi0_est, jets_est)
            
            #on verifie ici aussi la convergence
            if np.abs(lLikelihood[iCount] - prev_likelihood) < stopping_criterion:
                break
            prev_likelihood = lLikelihood[iCount]

            #calcul des probab de Viterbi pour les etats caches
            delta = np.zeros((len(jets_est), nS))
            psi = np.zeros((len(jets_est), nS), dtype=int)
            for t in range(len(jets_est)):
                for s in range(nS):
                    delta[t, s] = np.log(pi0_est[s]) + np.log(Ei_est[s, jets_est[t, 1]])

            #mise a jour des parametres Pij, Ei, pi0
            Pij_est = np.zeros((nS, nS))
            Ei_est = np.zeros((nS, nO))
            pi0_est = np.zeros(nS)
            
            for t in range(len(jets_est)-1):
                i = jets_est[t, 0]
                j = jets_est[t+1, 0]
                Pij_est[i, j] += 1
                Ei_est[i, jets_est[t, 1]] += 1

            pi0_est[jets_est[0, 0]] += 1
            
            #on normalise
            Pij_est /= Pij_est.sum(axis=1, keepdims=True)
            Ei_est /= Ei_est.sum(axis=1, keepdims=True)
            pi0_est /= pi0_est.sum()

            iCount += 1
        
        #on sauvegarde les meilleurs parametres en fonction de la log-vraisemblance
        if lLikelihood[iCount-1] > lLikelihood_best:
            Pij_best = Pij_est
            Ei_best = Ei_est
            pi0_best = pi0_est
            lLikelihood_best = lLikelihood[iCount-1]

    return Pij_best, Ei_best, pi0_best, lLikelihood_best

#on execute la fonction du Viterbi Training version 2.0 + affichage des resultats
nIterat = 100
Pij_best, Ei_best, pi0_best, lLikelihood_best = TrainingV2(jetsRes, 2, 2, nIterat)

print('Meilleur Pij estimée:')
print(Pij_best)
print('\nMeilleur Eij estimée:')
print(Ei_best)
print('\nMeilleur pi0 estimée:')
print(pi0_best)
print('\nMeilleure log-vraisemblance:')
print(lLikelihood_best)


C:\Users\maher\AppData\Local\Temp\ipykernel_54348\435899875.py:72: RuntimeWarning: divide by zero encountered in log
  delta[t, s] = np.log(pi0_est[s]) + np.log(Ei_est[s, jets_est[t, 1]])


Meilleur Pij estimée:
[[0.98956522 0.01043478]
 [0.06569343 0.93430657]]

Meilleur Eij estimée:
[[0.48521739 0.51478261]
 [0.91970803 0.08029197]]

Meilleur pi0 estimée:
[1. 0.]

Meilleure log-vraisemblance:
-1438.6637915308331


<font color="blue">
Remark: 
    - On remarque tout d'abord une convergence vers les mêmes valeurs (Pij, Eij et Pi(0) obtenues sont identiques à la première version de Viterbi malgré l'initialisation aléatoire faite dans la Viterbi 2. De même, les paramètres ne varient pas à travers les initialisations aléatoires ce qui suggère que le modèle est relativement robuste ici et que la convergence vers la solution optimale est fiable.
    - Après 100 tentatives on a une Log-vraisemblance de -1440 ce qui indique que le modèle est bien ajusté aux données.
    - Malgré les différentes initialisation aléatoires (ce qui permet de se défaire du biais de maximul local), le modèle converge toujours vers la même solution ce qui suggère un maximum global.
</font>